<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Code Functionality  Evals</h1>


This tests whether code is written correctly, without bugs, accomplishes the functionality you want, does not have syntax errors.

The purpose of this notebook is:

- to evaluate the performance of code fuctionality Eval
- to provide an experimental framework for users to iterate and improve on the default classification template.

## Install Dependencies and Import Libraries

In [ ]:
!pip install -qq arize-phoenix  "openai>=1" ipython matplotlib pycm scikit-learn tiktoken nest_asyncio 'httpx<0.28'

ℹ️ To enable async request submission in notebook environments like Jupyter or Google Colab, optionally use `nest_asyncio`. `nest_asyncio` globally patches `asyncio` to enable event loops to be re-entrant. This is not required for non-notebook environments.

Without `nest_asyncio`, eval submission can be much slower, depending on your organization's rate limits. Speed increases of about 5x are typical.

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

import matplotlib.pyplot as plt
import pandas as pd
from pycm import ConfusionMatrix
from sklearn.metrics import classification_report

from phoenix.evals import (
    # To Add templates
    OpenAIModel,
    llm_classify,
    create_classifier,
    evaluate_dataframe,
)

pd.set_option("display.max_colwidth", None)

## Download Benchmark Dataset

In [ ]:
df = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/phoenix/evals/code-functionality/validated_python_code_samples_2.csv"
)

df.head()


## Display Code Functionality Classification Template

View the default template used to code functionality. You can tweak this template and evaluate its performance relative to the default.

In [ ]:
CODE_FUNCTIONALITY_PROMPT_TEMPLATE = """
Code Evaluation Prompt:
-----------------------
Evaluate the provided code to determine its correctness in solving the given instruction.

Data:
-----
[Instruction]: {coding_instruction}
  Clearly define the task or problem that the code aims to address.

[Reference Code]: {code}
  Examine the submitted code for evaluation in the context of the provided instruction.

Evaluation:
-----------
Provide a concise response with a single word: either "bug_free" or "is_bug".
- "bug_free" signifies that the code correctly and efficiently solves the instruction with no bugs.
- "is_bug" indicates that the code either fails the instruction requirements or contains bugs.

Example:
-----------

[Instruction]: Implement the Fibonacci sequence in Python.

[Reference Code]: 'def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return
fibonacci(n - 1) + fibonacci(n - 2)\n\nfor i in range(10):\n    print(fibonacci(i))'

[Output]: bug_free

Note: Assumptions can be made that any code needed for the instruction is correct, and optimization
is not a requirement for a correct solution. Your response should consist solely of the words
"bug_free" or "is_bug" without additional text or characters.
"""

CODE_FUNCTIONALITY_PROMPT_RAILS_MAP = {'bug_free': 1, 'is_bug': 0}

The template variables are:

- **coding_instruction:** What is the code supposed to do as an instruction
- **code:** The code to evaluate 


## Configure the LLM

Configure your OpenAI API key.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

## Benchmark Dataset Sample
Sample size determines run time
Recommend iterating small: 100 samples
Then increasing to large test set

## LLM Evals: Code Functionality Classifications GPT-4
Run Code Functionality against a subset of the data.
Instantiate the LLM and set parameters.

In [ ]:
from phoenix.evals.llm import LLM

model = LLM(provider="openai", model="gpt-4")

## Run Code Func Classifications

Run code functionality classifications against a subset of the data.

In [ ]:
code_functionality_eval = create_classifier(
    name="code functionality",
    prompt_template=CODE_FUNCTIONALITY_PROMPT_TEMPLATE,
    llm=model,
    choices={'bug_free': 1, 'is_bug': 0}
)

results_df = evaluate_dataframe(dataframe=df, evaluators=[code_functionality_eval])

allLabels = results_df["code functionality_score"].apply(
        lambda x: x.get("label") if isinstance(x, dict) else None
)

allLabels.tolist()

## Evaluate Classifications

Evaluate the predictions against human-labeled ground-truth code functionality labels.

In [ ]:
true_labels = df["is_well_coded"].map(CODE_FUNCTIONALITY_PROMPT_RAILS_MAP).tolist()

print(classification_report(true_labels, allLabels, labels=["bug_free", "is_bug"]))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=allLabels, classes=["bug_free", "is_bug"]
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
)

## Classifications with explanations

When evaluating a dataset for code functionality, it can be useful to know why the LLM classified a document as relevant or irrelevant. The following code block runs `llm_classify` with explanations turned on so that we can inspect why the LLM made the classification it did. There is speed tradeoff since more tokens is being generated but it can be highly informative when troubleshooting.

In [ ]:
CODE_FUNCTIONALITY_PROMPT_TEMPLATE_WITH_EXPLANATION = """
Code Evaluation Prompt:
-----------------------
Evaluate the provided code to determine its correctness in solving the given instruction.

Data:
-----
[Instruction]: {coding_instruction}
  Clearly define the task or problem that the code aims to address.

[Reference Code]: {code}
  Examine the submitted code for evaluation in the context of the provided instruction.

Evaluation:
-----------
Provide a concise response with a explanation and a single word LABEL: either "bug_free" or
"is_bug".
- "bug_free" signifies that the code correctly and efficiently solves the instruction with no bugs.
- "is_bug" indicates that the code either fails to meet the instruction requirements or contains
bugs.

Example:
-----------

[Instruction]: Implement the Fibonacci sequence in Python.

[Reference Code]: 'def fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return
fibonacci(n - 1) + fibonacci(n - 2)\n\nfor i in range(10):\n    print(fibonacci(i))'

[Output]: bug_free

Note: Assumptions can be made that any code needed for the instruction is correct, and optimization
is not a requirement for a correct solution. Your response should consist solely of the words
"bug_free" or "is_bug" without additional text or characters.

Please read the instruction and code carefully, then write out in a step by step manner an
EXPLANATION to show how to evaluate the functionality of the code. Avoid simply stating the correct
answer at the outset.
You are then going to respond with a LABEL (a single word evaluation).
If the reference code functionally solves the instruction problem without any bugs than call it
"bug_free".
If reference code has bugs or does not functionally solve the instruction problem than call it
"is_bug".

Example response:
************
EXPLANATION: An explanation of your reasoning for why the code is bug_free or is_bug
LABEL: "bug_free" or "is_bug"
************

EXPLANATION:
"""

In [ ]:
small_df_sample = df.copy().sample(n=5).reset_index(drop=True)


code_functionality_eval = create_classifier(
    name="code functionality",
    prompt_template=CODE_FUNCTIONALITY_PROMPT_TEMPLATE_WITH_EXPLANATION,
    llm=model,
    choices={'bug_free': 1, 'is_bug': 0}
)

results_df = evaluate_dataframe(dataframe=small_df_sample, evaluators=[code_functionality_eval])

In [ ]:
# Let's view the data
merged_df = pd.merge(
    small_df_sample, results_df, left_index=True, right_index=True
)
merged_df[["coding_instruction", "code", "label", "explanation"]].head()

## LLM Evals: code functionality Classifications GPT-3.5 Turbo
Run Code functionality against a subset of the data using GPT-3.5. GPT-3.5 can significantly speed up the classification process. However there are tradeoffs as  we will see below.

In [ ]:
model = LLM(provider="openai", model="gpt-3.5-turbo")

In [ ]:
code_functionality_eval = create_classifier(
    name="code functionality",
    prompt_template=CODE_FUNCTIONALITY_PROMPT_TEMPLATE,
    llm=model,
    choices={'bug_free': 1, 'is_bug': 0}
)

relevance_classifications = evaluate_dataframe(dataframe=df, evaluators=[code_functionality_eval])

relevance_classifications = relevance_classifications["code functionality_score"].apply(
        lambda x: x.get("label") if isinstance(x, dict) else None
)

relevance_classifications.tolist()

In [ ]:
true_labels = df["is_well_coded"].map(CODE_FUNCTIONALITY_PROMPT_RAILS_MAP).tolist()

print(classification_report(true_labels, relevance_classifications, labels=["bug_free", "is_bug"]))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=relevance_classifications, classes=["bug_free", "is_bug"]
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
)

## Preview: Running with GPT-4 Turbo

In [ ]:
model = LLM(provider="openai", model="gpt-4-1106-preview")

code_functionality_eval = create_classifier(
    name="code functionality",
    prompt_template=CODE_FUNCTIONALITY_PROMPT_TEMPLATE,
    llm=model,
    choices={'bug_free': 1, 'is_bug': 0}
)

classifications = evaluate_dataframe(dataframe=df, evaluators=[code_functionality_eval])
classifications = classifications["code functionality_score"].apply(
        lambda x: x.get("label") if isinstance(x, dict) else None
)

classifications.tolist()

In [ ]:
true_labels = df["is_well_coded"].map(CODE_FUNCTIONALITY_PROMPT_RAILS_MAP).tolist()

print(classification_report(true_labels, classifications, labels=["bug_free", "is_bug"]))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=classifications, classes=["bug_free", "is_bug"]
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
)